# Credit Analytics - Final Project Code
By: Dennis Goldenberg, Triet Vo, Kezia 

In [28]:
import numpy as np
import pandas as pd
from scipy.stats import norm, binom

C:\Users\Dennis Goldenberg\AppData\Local\Temp\ipykernel_21048\3981726678.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 1. Comparing Binomial Expansion Model to Gaussian Copula Model

In [29]:
num_assets = 1000
Notional = 100000000

#Generate default probabilities - 250 names rated BBB, BB, B, and C
rd_probs = [.0243, .0811, .1104, .389]
default_probs = [.0243] * int(num_assets/4) + [.0811]*int(num_assets/4)
default_probs = default_probs + [.1104]*int(num_assets/4) + [.389]*int(num_assets/4)
default_probs = np.asarray(default_probs)

#recover number of industries given default rates
N_i = np.asarray([int(15/.0243),int(8/.0811),int(16/.1105), int(3/.389)])
D_i = np.asarray([int(15), int(8), int(16),int(3)])


#calculate joint probabilities and default correlation
joint_prob = np.empty(shape = (4,4))
corr_mat = np.empty(shape = (4,4))
for i in range(len(N_i)):
    for j in range(len(N_i)):
        if i == j:
            joint_prob[i,j] = (D_i[i]**2)/(N_i[i]**2)
            corr_mat[i,j] = (joint_prob[i,j] - (rd_probs[i]**2))/(rd_probs[i]*(1 - rd_probs[i]))
        else:
            joint_prob[i,j] = (D_i[i] * D_i[j])/(N_i[i]*N_i[j])
            corr_mat[i,j] = (joint_prob[i,j] - (rd_probs[i]*rd_probs[j]))
            corr_mat[i,j] = corr_mat[i,j]/np.sqrt(rd_probs[i]*(1 - rd_probs[i]) * rd_probs[j]*(1 - rd_probs[j]))

The joint probability of defaults is shown below:

In [33]:
ratings = ["BBB", "BB", "B", "C"]
jp_df = pd.DataFrame(data = joint_prob, columns = ratings, index = ratings)
jp_df

,BBB,BB,B,C
BBB,0.000591,0.001985,0.002701,0.010419
BB,0.001985,0.006664,0.009070,0.034985
B,0.002701,0.009070,0.012346,0.047619
C,0.010419,0.034985,0.047619,0.183673


Next, the default correlation between two names, depending on their rating, is shown below:

In [34]:
rho_df = pd.DataFrame(data = corr_mat, columns = ratings, index = ratings)
rho_df

,BBB,BB,B,C
BBB,0.000023,0.000330,0.000384,0.012873
BB,0.000330,0.001163,0.001366,0.025829
B,0.000384,0.001366,0.001604,0.030589
C,0.012873,0.025829,0.030589,0.136118


## 2. Comparing Gaussian Copula to Fudged Period by Period Copula